#### 실시간 번역 + TTS

In [ ]:
import os
import time
import threading
from dotenv import load_dotenv
import azure.cognitiveservices.speech as speechsdk
from datetime import datetime

# 환경변수 로드
load_dotenv()

speech_api_key = os.getenv("SPEECH_API_KEY")
region = os.getenv("SPEECH_REGION")

# ==================== 언어 설정 ====================
INPUT_LANGUAGE = "영어"      # STT 입력 언어  
OUTPUT_LANGUAGE = "한국어"    # 번역 출력 + TTS 출력 언어

# 언어 매핑 (실시간 번역용)
SPEECH_TRANSLATION_MAPPING = {
    "한국어": ("ko-KR", "ko", "ko-KR-SunHiNeural"),
    "영어": ("en-US", "en", "en-US-AvaMultilingualNeural"),
    "중국어": ("zh-CN", "zh-Hans", "zh-CN-XiaoxiaoNeural"),
    "일본어": ("ja-JP", "ja", "ja-JP-NanamiNeural"),
    "독일어": ("de-DE", "de", "de-DE-KatjaNeural"),
    "프랑스어": ("fr-FR", "fr", "fr-FR-DeniseNeural"),
    "스페인어": ("es-ES", "es", "es-ES-ElviraNeural"),
    "이탈리아어": ("it-IT", "it", "it-IT-IsabellaNeural"),
    "러시아어": ("ru-RU", "ru", "ru-RU-DariyaNeural"),
    "포르투갈어": ("pt-BR", "pt", "pt-BR-FranciscaNeural")
}

INPUT_STT_CODE, INPUT_TRANSLATE_CODE, INPUT_TTS_VOICE = SPEECH_TRANSLATION_MAPPING[INPUT_LANGUAGE]
OUTPUT_STT_CODE, OUTPUT_TRANSLATE_CODE, OUTPUT_TTS_VOICE = SPEECH_TRANSLATION_MAPPING[OUTPUT_LANGUAGE]

print(f"🎤 입력 언어: {INPUT_LANGUAGE} ({INPUT_STT_CODE})")
print(f"🔄 출력 언어: {OUTPUT_LANGUAGE} ({OUTPUT_TRANSLATE_CODE})")
print("=" * 50)

# ==================== 실시간 번역 클래스 ====================
class AzureSpeechTranslation:
    def __init__(self):
        # Speech Translation 설정
        self.translation_config = speechsdk.translation.SpeechTranslationConfig(
            subscription=speech_api_key, 
            region=region
        )
        self.translation_config.speech_recognition_language = INPUT_STT_CODE
        self.translation_config.add_target_language(OUTPUT_TRANSLATE_CODE)
        
        # TTS 설정 (기존 코드 활용)
        self.tts_config = speechsdk.SpeechConfig(subscription=speech_api_key, region=region)
        self.tts_config.speech_synthesis_voice_name = OUTPUT_TTS_VOICE
        
        print("✅ Azure Speech Translation 초기화 완료")
    
    def translate_once(self):
        """한 번 음성을 인식하고 번역"""
        audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
        translation_recognizer = speechsdk.translation.TranslationRecognizer(
            translation_config=self.translation_config, 
            audio_config=audio_config
        )
        
        print("🎤 말씀하세요...")
        result = translation_recognizer.recognize_once_async().get()
        
        if result.reason == speechsdk.ResultReason.TranslatedSpeech:
            original_text = result.text
            translated_text = result.translations.get(OUTPUT_TRANSLATE_CODE, "")
            
            print(f"✅ 원본 ({INPUT_LANGUAGE}): {original_text}")
            print(f"🔄 번역 ({OUTPUT_LANGUAGE}): {translated_text}")
            
            # TTS 출력
            if translated_text:
                self.speak_text(translated_text)
            
            return {
                "original": original_text,
                "translated": translated_text,
                "success": True
            }
            
        elif result.reason == speechsdk.ResultReason.NoMatch:
            print("❌ 음성을 인식할 수 없습니다.")
            return {"success": False, "error": "No speech recognized"}
            
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            print(f"❌ 인식 취소됨: {cancellation_details.reason}")
            if cancellation_details.reason == speechsdk.CancellationReason.Error:
                print(f"오류 상세: {cancellation_details.error_details}")
            return {"success": False, "error": str(cancellation_details.reason)}
    
    def continuous_translation(self):
        """연속 실시간 번역 (사용자가 종료하기 전까지 계속)"""
        print(f"🎤 {INPUT_LANGUAGE} → {OUTPUT_LANGUAGE} 실시간 번역 시작")
        print("말씀하시면 자동으로 번역되고 음성 출력됩니다.")
        print("종료하려면 'q' 또는 'quit'을 입력하세요.")
        print("-" * 50)
        
        audio_config = speechsdk.audio.AudioConfig(use_default_microphone=True)
        translation_recognizer = speechsdk.translation.TranslationRecognizer(
            translation_config=self.translation_config,
            audio_config=audio_config
        )
        
        # 인식 완료 플래그
        done = False
        
        def recognized_handler(evt):
            """번역 완료 시 자동으로 TTS 실행"""
            if evt.result.reason == speechsdk.ResultReason.TranslatedSpeech:
                original_text = evt.result.text
                translated_text = evt.result.translations.get(OUTPUT_TRANSLATE_CODE, "")
                
                if original_text and translated_text:
                    print(f"\n✅ 원본: {original_text}")
                    print(f"🔄 번역: {translated_text}")
                    
                    # 자동 TTS
                    self.speak_text(translated_text)
                    print("-" * 30)
        
        def recognizing_handler(evt):
            """실시간 번역 중 (부분 결과)"""
            if evt.result.reason == speechsdk.ResultReason.TranslatingSpeech:
                original_text = evt.result.text
                translated_text = evt.result.translations.get(OUTPUT_TRANSLATE_CODE, "")
                if original_text:
                    print(f"🎤 인식중: {original_text}", end='\r')
        
        def canceled_handler(evt):
            """취소/오류 핸들러"""
            nonlocal done
            print(f"❌ 번역 취소/오류: {evt}")
            done = True
        
        def stopped_handler(evt):
            """세션 종료 핸들러"""
            nonlocal done
            print(f"🛑 세션 종료: {evt}")
            done = True
        
        # 올바른 이벤트 핸들러 연결
        translation_recognizer.recognized.connect(recognized_handler)    # translated → recognized
        translation_recognizer.recognizing.connect(recognizing_handler)  # translating → recognizing  
        translation_recognizer.canceled.connect(canceled_handler)
        translation_recognizer.session_stopped.connect(stopped_handler)
        
        # 연속 번역 시작
        translation_recognizer.start_continuous_recognition_async()
        
        # 사용자 입력 대기 (별도 스레드)
        def wait_for_user_input():
            nonlocal done
            while not done:
                try:
                    user_input = input().strip().lower()
                    if user_input in ['q', 'quit', 'exit', '종료']:
                        print("👋 사용자가 종료를 요청했습니다.")
                        done = True
                        break
                except (EOFError, KeyboardInterrupt):
                    print("\n👋 키보드 인터럽트로 종료합니다.")
                    done = True
                    break
        
        # 입력 대기 스레드 시작
        input_thread = threading.Thread(target=wait_for_user_input, daemon=True)
        input_thread.start()
        
        # 메인 루프
        try:
            while not done:
                time.sleep(0.5)
        except KeyboardInterrupt:
            print("\n👋 키보드 인터럽트로 종료합니다.")
            done = True
        finally:
            print("🛑 실시간 번역을 중지합니다...")
            translation_recognizer.stop_continuous_recognition_async()
    
    def speak_text(self, text):
        """기존 TTS 코드 활용 - 스피커 출력"""
        try:
            audio_config = speechsdk.audio.AudioOutputConfig(use_default_speaker=True)
            synthesizer = speechsdk.SpeechSynthesizer(self.tts_config, audio_config)
            
            result = synthesizer.speak_text_async(text).get()
            
            if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
                print(f"🔊 [{OUTPUT_LANGUAGE}] TTS 출력 완료")
                return True
            else:
                print(f"❌ TTS 실패: {result.cancellation_details.reason}")
                return False
        except Exception as e:
            print(f"❌ TTS 오류: {e}")
            return False

# ==================== 메인 실행부 ====================
def main():
    print("🎯 Azure 실시간 음성 번역 + TTS 시스템")
    print("=" * 50)
    print("1. 한 번 번역 (recognize_once)")
    print("2. 실시간 연속 번역 (continuous)")
    print("0. 종료")
    
    translator = AzureSpeechTranslation()
    
    while True:
        choice = input("\n선택하세요 > ").strip()
        
        if choice == "1":
            # 한 번 번역
            result = translator.translate_once()
            
        elif choice == "2":
            # 실시간 연속 번역
            translator.continuous_translation()
            
        elif choice == "0":
            print("👋 프로그램을 종료합니다.")
            break
            
        else:
            print("❌ 잘못된 선택입니다.")

if __name__ == "__main__":
    main()
